### Задача

Подготовить исследование рынка по открытым данным о заведениях общественного питания в Москве. Определить популярность в долгосрочной перспективе нового кафе. Проект отличается от других заведений идеей обслуживания гостей роботами.

### Оглавление

[1. Открытие и изучение данных](#1)  
[2. Анализ данных](#2)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Исследуйте соотношение видов объектов общественного питания по количеству. Постройте график.](#2.1)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Исследуйте соотношение сетевых и несетевых заведений по количеству. Постройте график.](#2.2)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Для какого вида объекта общественного питания характерно сетевое распространение?](#2.3)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 Что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест?](#2.4)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.5 Для каждого вида объекта общественного питания опишите среднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест? Постройте графики.](#2.5)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.6 Постройте график топ-10 улиц по количеству объектов общественного питания. Воспользуйтесь внешней информацией и ответьте на вопрос — в каких районах Москвы находятся эти улицы?](#2.6)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.7 Найдите число улиц с одним объектом общественного питания. Воспользуйтесь внешней информацией и ответьте на вопрос — в каких районах Москвы находятся эти улицы?](#2.7)  
&nbsp;&nbsp;&nbsp;&nbsp;[2.8 Посмотрите на распределение количества посадочных мест для улиц с большим количеством объектов общественного питания. Какие закономерности можно выявить?](#2.8)  
[3. Вывод](#3)

### Импорт библиотек и настройки

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go

warnings.filterwarnings('ignore')
sns.set(style='whitegrid')
# цветовая палитра, с которой будем работать
colors = ['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', 
          '#0099C6', '#DD4477','#66AA00', '#B82E2E', '#316395']

ModuleNotFoundError: No module named 'plotly'

### 1. Открытие и изучение данных <a id='1'></a>

In [ ]:
rest_data = pd.read_csv('/datasets/rest_data.csv')
rest_data.head(5)

In [ ]:
rest_data.info()

Названия колонок корректны, типы данных правильные, пропущенных значений нет.

Проверка наличия дубликатов

In [ ]:
rest_data.duplicated().sum()

### 2. Анализ данных <a id='2'></a>

#### 2.1 Исследуйте соотношение видов объектов общественного питания по количеству. Постройте график. <a id='2.1'></a>

In [ ]:
count_object = rest_data.groupby('object_type')['id']\
                        .count()\
                        .reset_index()\
                        .sort_values(by='id', ascending=False)
count_object.head(5)

In [ ]:
fig = plt.figure(figsize=(10, 8))
sns.barplot(data=count_object, x='id', y='object_type', palette=colors) 
plt.title('Количество заведений по типам')
plt.xlabel('count')

По количеству выделяется кафе. Значит больше всего спрос именно на этот тип заведений. Далее значительное количество у таких типов, как столовая, ресторан и предприятие быстрого обслуживания.

#### 2.2 Исследуйте соотношение сетевых и несетевых заведений по количеству. Постройте график. <a id='2.2'></a>

In [ ]:
rest_data.groupby('object_type')['chain'].value_counts(normalize=True).map(lambda x: '{:.0%}'.format(x))

Преобладают несетевые заведения. В кафе 77% несетевых и 23% сетевых.

Создадим датафрейм с количеством заведений и построим график.

In [ ]:
count_chain_object = rest_data.groupby(['object_type','chain'])['id']\
            .count()\
            .reset_index()\
            .sort_values(by='id', ascending=False)
count_chain_object.head(5)

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='id',
            y='object_type', 
            hue='chain',
            data=count_chain_object,
           palette=colors)
plt.title('Количество заведений по типам')
plt.xlabel('count')

В Москве преобладают отдельные, несетевые заведения. Поэтому топ несетевых заведений совпадает с общим. А в топе сетевых заведений - кафе, предприятие быстрого обслуживания и ресторан.

#### 2.3 Для какого вида объекта общественного питания характерно сетевое распространение? <a id='2.3'></a>

In [ ]:
chain_obj = count_chain_object[count_chain_object['chain'] == 'да']
chain_obj['id'] = (
    chain_obj['id'] / (chain_obj['id'].sum())
).map(lambda x: '{:.0%}'.format(x))
chain_obj

In [ ]:
fig = plt.figure(figsize=(10, 8))
sns.barplot(data=count_chain_object[count_chain_object['chain'] == 'да'], 
            x='id', 
            y='object_type', 
            palette=colors) 
plt.title('Сетевые заведения по типам')
plt.xlabel('count')

Почти 1400 сетевых заведений - это кафе (47% от всего числа заведений). Предприятие быстрого обслуживания - около 800 заведений (27%), ресторан - около 550 (18%).

#### 2.4 Что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест? <a id='2.4'></a>

Создадим датафрейм с количеством заведений в сети и количеством посадочных мест.

In [ ]:
rest_chain_data = rest_data[rest_data['chain'] == 'да']\
                  .groupby('object_name')\
                  .agg({'id':'count', 'number':'mean'})\
                  .reset_index()
rest_chain_data.columns = ['object_name', 'count_object', 'mean_number']
rest_chain_data.head(5)

Рассчитаем основные характеристики выборок.

In [ ]:
rest_chain_data.describe()

In [ ]:
sns.jointplot(x='mean_number', y='count_object', data=rest_chain_data, color='#3366CC')

Для большей части сетевых заведений характерно малое число заведений (не более 5) с большим количеством посадочных мест (в среднем 45).

#### 2.5 Для каждого вида объекта общественного питания опишите среднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест? Постройте графики. <a id='2.5'></a>

Расчитаем медианные значения количества посадочных мест для каждого типа заведения.

In [ ]:
median_number = rest_data.groupby('object_type')\
.agg({'number':'median'})\
.reset_index()\
.sort_values(by='number', ascending=False)
median_number

In [ ]:
fig = plt.figure(figsize=(18, 10))
sns.boxplot(x='number', 
            y='object_type', 
            data=rest_data,
            order=median_number['object_type'].tolist(), 
            showfliers = False, 
            palette=colors)
plt.title('Количество посадочных мест')
plt.show()

Самое большое количество мест в столовой (в среднем около 100), также большое количество мест в ресторане (около 80). Кафе, буфет и бар близки по средним значениям (30-35). Другие виды заведений имеют преимущественно малое количество мест, либо они отсутствуют.

#### 2.6 Постройте график топ-10 улиц по количеству объектов общественного питания. Воспользуйтесь внешней информацией и ответьте на вопрос — в каких районах Москвы находятся эти улицы? <a id='2.6'></a>

Выделим в отдельный столбец информацию об улице из столбца `address`.

In [ ]:
rest_data['street'] = rest_data['address'].str.split(pat=', ',expand=True)[1]

Подгружаем таблицу с адресами Москвы.

In [ ]:
from io import BytesIO
import requests
spreadsheet_id = '1lC6pukCjj7bdBNiBWqDqAZX3R3ojI5t-YBAoorK2YM8'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=xlsx'.format(spreadsheet_id)
r = requests.get(file_name)
moscow = pd.read_excel(BytesIO(r.content))
moscow = moscow[['address', 'district', 'street']]
moscow.head(5)

По адресу соединим датафреймы для определения района.

In [ ]:
data = rest_data.merge(moscow[['address', 'district']], on='address', how='left')
data

Посмотрим количество неопределенных районов.

In [ ]:
len(data[data['district'].isna()])

In [ ]:
na_district = data[data['district'].isna()]
na_district.head()

Дозаполним пропуски. Для неопределенных адресов будем использовать соответствие района по улице. 

In [ ]:
for index, street in na_district['street'].items():
    for index_el, el in moscow['street'].items():
        if street == el:
            data['district'][index] = moscow['district'][index_el]
            break    

In [ ]:
print('Количество данных без района -', len(data[data['district'].isna()]))
print('Район отсутствует в {:.2%} случаях'.format(len(data[data['district'].isna()]) / len(data)))

Район не определился в меньше 1% случаев. Удалим данные, так как значительного влияния на выводы это не окажет.

In [ ]:
data.dropna(subset=['district'], inplace=True)
print('Количество данных без района -', len(data[data['district'].isna()]))

Самые популярные улицы.

In [ ]:
top_street = data.groupby(['street'])['id'].count().sort_values(ascending=False).head(12).reset_index()
top_street

Удалим города и поселения.

In [ ]:
top_street.drop(top_street.index[[0, 7]], inplace=True)
top_street = top_street.reset_index(drop=True)
top_street

Построим график топ-10 улиц по количеству объектов общественного питания.

In [ ]:
fig = plt.figure(figsize=(10, 8))
sns.barplot(data=top_street, x='id', y='street', palette=colors) 
plt.title('Количество заведений на улицах')
plt.xlabel('count')

In [ ]:
data[data['street'].isin(top_street['street'].tolist())]['district'].value_counts().head(10)

Самые популярные районы.

In [ ]:
top_district = data.groupby(['district'])['id'].count().sort_values(ascending=False).head(10).reset_index()
top_district

Построим график топ-10 районов по количеству объектов общественного питания.

In [ ]:
fig = plt.figure(figsize=(10, 8))
sns.barplot(data=top_district, x='id', y='district', palette=colors) 
plt.title('Количество заведений в районах')
plt.xlabel('count')

Районы по топ-10 улиц по большей части отличаются от топ-10 районов. Концентрация заведений в определенных районах больше, чем концентрация заведений по конкретным улицам, которые входят в сразу несколько районов. Поэтому для районов будем ориентироваться не на топ-10 улиц, а на топ-10 районов.

#### 2.7 Найдите число улиц с одним объектом общественного питания. Воспользуйтесь внешней информацией и ответьте на вопрос — в каких районах Москвы находятся эти улицы? <a id='2.7'></a>

In [ ]:
street_with_1object = data.groupby('street')['id'].count().sort_values().reset_index()
street_with_1object = street_with_1object[street_with_1object['id'] == 1]
print('Количество улиц с одним объектом общественного питания -',len(street_with_1object))
street_with_1object.head(10)

Найдем районы, включающие в себя улицы с одним объектом общественного питания.

In [ ]:
distr_with_1object = data[data['street'].isin(street_with_1object['street'].tolist())]['district']\
                     .value_counts().reset_index()
distr_with_1object.columns = ['district', 'count_street']
distr_with_1object.head(10)

Популярные районы тоже попали в список. Они тоже содержат переулки и улицы только с 1 объектом.

#### 2.8 Посмотрите на распределение количества посадочных мест для улиц с большим количеством объектов общественного питания. Какие закономерности можно выявить? <a id='2.8'></a>

In [ ]:
number_top_street = data[data['street'].isin(top_street['street'].tolist())]['number']
number_top_street.head(10)

In [ ]:
number_top_street.describe()

In [ ]:
fig = plt.figure(figsize=(12, 7))
sns.distplot(number_top_street, bins=90, color='#3366CC') 
plt.title('Количество мест на популярных улицах')
plt.xlabel('count')
plt.xlim(0,500)

Распределение нормальное с пиком около 15. В среднем, количество мест на популярных улицах(40) на 5 меньше, чем количество мест на всех улицах (45). 

<a class="anchor" id="0-bullet">

### Вывод <a id='3'></a>

По результатам анализа можно выделить следующие характеристики:  
* самый часто встречаемый вид заведения - это кафе, значит кафе пользуются большей популярностью;
* несетевых заведений по Москве больше, по кафе отмечено 77% несетевых и 23% сетевых;
* количество посадочных мест, в среднем для кафе - 30, самое частое количество - около 20;  
* на улицах и в районах с самым большим количеством заведений большая проходимость людей и спрос, поэтому лучше выбрать из таких популярных мест.  
ТОП-10 районов: Тверской, Пресненский, Басманный, Даниловский, Замоскворечье, Хамовники, Мещанский, Таганский, Арбат, Хорошевский.  
ТОП-10 улиц: проспект Мира, Профсоюзная улица, Ленинградский проспект, Пресненская набережная, Варшавское шоссе, Ленинский проспект, проспект Вернадского, Кутузовский проспект, Каширское шоссе, Кировоградская улица.

Презентация: <https://drive.google.com/file/d/1ljC45MWJv3-BVFF5YrtsedqkdMG_LvsF/view?usp=sharing>